In [1]:
import os
import sys

sys.path.append(os.path.abspath('..'))

In [2]:
from typing import Dict, List, Optional, Any
from datetime import datetime

import httpx
from loguru import logger
from smolagents import CodeAgent, LiteLLMModel, Tool, PromptTemplates
import yaml

import src.agent.adapters.tools  as tools

In [8]:
api_base=None

# model_id="ollama/qwen3:4b"
# api_base="http://localhost:11434"

# anthorpic
# model_id="claude-3-7-sonnet-20250219"

#openai
model_id = "gpt-4o"

# google
# model_id = "gemini/gemini-2.0-flash-exp"
# model_id = "gemin/gemini-2.5-flash-preview-04-17"
# model_id = "gemini/gemini-2.5-pro-preview-05-06"

kwargs = {"tools_api_base": "http://localhost:5050",
          "tools_api_limit": 100}

In [9]:
model = LiteLLMModel(model_id=model_id, api_base=api_base)

In [10]:
with open("/Users/steffen/agentic_ai/src/agent/prompts/base_prompts.yaml", "r") as file:
    base_prompts = yaml.safe_load(file)


base_prompts['system_prompt'] = base_prompts['system_prompt'].replace("{{current_date}}", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
prompt_templates = PromptTemplates(**base_prompts)

In [11]:
class FinalAnswerTool(Tool):
    name = "final_answer"
    description = "Provides a final answer to the given problem."
    inputs = {"answer": {"type": "any", "description": "The final answer to the problem"}}
    outputs = {"result": {"type": "any", "description": "The final response to the problem"}}
    output_type = "any"

    def forward(self, answer: Any) -> Any:
        return answer


In [12]:
agent = CodeAgent(tools=[tools.CompareData(**kwargs), 
                         tools.ConvertIdToName(**kwargs),
                         tools.ConvertNameToId(**kwargs),
                         tools.GetData(**kwargs),
                         tools.GetInformation(**kwargs),
                         tools.GetNeighbors(**kwargs),
                         tools.PlotData(**kwargs),
                         FinalAnswerTool()
                        ],  
                  model=model, 
                  stream_outputs=True,
                  additional_authorized_imports=["pandas", "numpy"],
                  prompt_templates=prompt_templates,
                  max_steps=5)

In [79]:
result = agent.run("What is the daily maximum value of PI-P0017 in April 2025?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the daily maximum value of PI-P0017 in April 2025?                                                      │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o ─────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output()

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Get the asset id for PI-P0017                                                                                  
  out = name_to_id(["PI-P0017"])                                                                                   
  print(out)                                                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

2025-05-21 16:10:55.723 | INFO     | src.agent.adapters.tools.base:call_api:74 - Fetching data from http://localhost:5050/v1/id_from_name/PI-P0017 with params: {'offset': 0, 'limit': 100}
2025-05-21 16:10:55.793 | INFO     | src.agent.adapters.tools.base:call_api:88 - Reached the last page.


...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output()

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(daily_maximum_values)                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer:                      18b04353-839d-40a1-84c1-9b547d09dd80
timestamp                                                
2025-04-01T00:00:00                              1.588889
2025-04-02T00:00:00                              1.556674
2025-04-03T00:00:00                              1.458451
2025-04-04T00:00:00                              1.481194
2025-04-05T00:00:00                              1.378924
2025-04-06T00:00:00                              1.267590
2025-04-07T00:00:00                              1.261139
2025-04-08T00:00:00                              1.290472
2025-04-09T00:00:00                              1.241875
2025-04-10T00:00:00                              1.178278
2025-04-11T00:00:00                              1.025847
2025-04-12T00:00:00                              1.014632
2025-04-13T00:00:00                              0.948687
2025-04-14T00:00:00                              0.888639
2025-04-15T00:00:00                              0.945736
2025-04-16T00:00:00                              0.842243
2025-04-17T00:00:00                              0.785375
2025-04-18T00:00:00                              0.804521
2025-04-19T00:00:00                              0.783035
2025-04-20T00:00:00                              0.699667
2025-04-21T00:00:00                              0.635833
2025-04-22T00:00:00                              0.625757
2025-04-23T00:00:00                              0.619875
2025-04-24T00:00:00                              0.515604
2025-04-25T00:00:00                              0.608785
2025-04-26T00:00:00                              0.500250
2025-04-27T00:00:00                              0.435174
2025-04-28T00:00:00                              0.363062
2025-04-29T00:00:00                              0.378806
2025-04-30T00:00:00                              0.355576

[Step 4: Duration 1.56 seconds| Input tokens: 10,961 | Output tokens: 383]

In [38]:
....__dir__()

['system_prompt',
 'steps',
 '__module__',
 '__init__',
 'reset',
 'get_succinct_steps',
 'get_full_steps',
 'replay',
 '__dict__',
 '__weakref__',
 '__doc__',
 '__new__',
 '__repr__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__reduce_ex__',
 '__reduce__',
 '__getstate__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [42]:
agent.memory.get_full_steps()[1].keys()

dict_keys(['model_input_messages', 'tool_calls', 'start_time', 'end_time', 'step', 'error', 'duration', 'model_output_message', 'model_output', 'observations', 'action_output'])

In [54]:
agent.memory.get_full_steps()[3]["tool_calls"]

[{'id': 'call_3',
  'type': 'function',
  'function': {'name': 'python_interpreter',
   'arguments': "# Step 3: Calculate the maximum value from the daily data\ndaily_max_value = data['data'].max().max()\nfinal_answer(daily_max_value)"}}]

In [67]:
from smolagents.memory import TaskStep, ActionStep, PlanningStep

In [77]:
temp = []

for step in agent.memory.steps:
    if type(step) == TaskStep:
        temp.append(step.task)
    elif type(step) == ActionStep:
        if step.model_output is not None:
            temp.append(step.model_output)
    elif type(step) ==PlanningStep:
        temp.append(step.plan)


In [66]:
type(agent.memory.steps[0])

smolagents.memory.TaskStep

In [59]:
abc = agent.memory.replay(agent.logger)

[15:37:41] Replaying the agent's steps:                                                               ]8;id=192163;file:///Users/steffen/miniconda3/envs/py312/lib/python3.12/site-packages/smolagents/memory.py\memory.py]8;;\:]8;id=89158;file:///Users/steffen/miniconda3/envs/py312/lib/python3.12/site-packages/smolagents/memory.py#212\212]8;;\

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the daily maximum value of PI-P0017 in April 2025?                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Agent output: ─────────────────────────────────────────────────────────────────────────────────────────────────────
Thought: I need to get the asset id of PI-P0017 first. Then I need to get the daily data for the received asset_id 
for April 2025. Finally, I will calculate the maximum value for each day.                                          
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
# Step 1: Get the asset id for PI-P0017                                                                            
out = name_to_id(["PI-P0017"])                                                                                     
print(out)                                                                                                         
```<end_code>                                                                                                      

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Agent output: ─────────────────────────────────────────────────────────────────────────────────────────────────────
Thought: Now that I have the asset id '18b04353-839d-40a1-84c1-9b547d09dd80' for PI-P0017, I need to get the daily 
aggregated data for April 2025. After obtaining the data, I will calculate the maximum value for each day.         
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
# Step 2: Get the daily data for the asset id in April 2025                                                        
start_date = "2025-04-01T00:00:00Z"                                                                                
end_date = "2025-04-30T23:59:59Z"                                                                                  
data = get_data(asset_ids=["18b04353-839d-40a1-84c1-9b547d09dd80"], start_date=start_date, end_date=end_date,      
aggregation="day", last_value=False)                                                                               
print(data)                                                                                                        
```<end_code>                                                                                                      

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Agent output: ─────────────────────────────────────────────────────────────────────────────────────────────────────
Thought: I have obtained the daily data for April 2025. Now, I need to find the maximum value from this data.      
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
# Step 3: Calculate the maximum value from the daily data                                                          
daily_max_value = data['data'].max().max()                                                                         
final_answer(daily_max_value)                                                                                      
```<end_code>                                                                                                      

In [55]:
agent.memory.__dir__()

['system_prompt',
 'steps',
 '__module__',
 '__init__',
 'reset',
 'get_succinct_steps',
 'get_full_steps',
 'replay',
 '__dict__',
 '__weakref__',
 '__doc__',
 '__new__',
 '__repr__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__reduce_ex__',
 '__reduce__',
 '__getstate__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [30]:
agent.extract_action.__dir__()

['__new__',
 '__repr__',
 '__hash__',
 '__call__',
 '__getattribute__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__reduce__',
 '__func__',
 '__self__',
 '__doc__',
 '__str__',
 '__setattr__',
 '__delattr__',
 '__init__',
 '__reduce_ex__',
 '__getstate__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

## Tests

In [ ]:
tasks = [
    "Can you compare PI-P0017 and PI-P0016 for the first 10 days in 2025?",
    "What assets are next to asset BA100?",
    "Can you create a plot for the adjacent sensors of asset BA101 for 1st January 2025?",
    "What is the id of TI-T0022?",
    "What is the name of asset id c831fadb-d620-4007-bdda-4593038c87f9?",
    "Can you provide me the highest value for June 2025 for TI-T0022?"
]

In [ ]:
result = agent.run(tasks[0])

In [ ]:
result = agent.run(tasks[1])

In [ ]:
result = agent.run(tasks[2])

In [ ]:
base64_string_to_decode = result["plot"]

In [ ]:
import matplotlib.image as mpimg

import base64
import io
import matplotlib.pyplot as plt


In [ ]:
image_bytes = base64.b64decode(base64_string_to_decode)

# 2. Create an in-memory buffer from the bytes
image_buffer = io.BytesIO(image_bytes)

# 3. Load Image from Buffer using Matplotlib
# mpimg.imread can read from a file-like object
img_data = mpimg.imread(image_buffer, format='png') # Specify format if known (e.g., 'png', 'jpeg')

# 4. Display Image using Matplotlib
plt.figure(figsize=(8, 6)) # Optional: Adjust figure size
plt.imshow(img_data)
plt.axis('off')  # Turn off axis numbers and ticks for a cleaner image display
plt.title("Decoded Image")
plt.show()


In [ ]:
result = agent.run(tasks[3])

In [ ]:
result = agent.run(tasks[4])

In [ ]:
result = agent.run(tasks[5])